Antes de empezar: un poco de documentación para enterder ligeramente cómo funciona HTML:

https://www.w3schools.com/html/html_intro.asp

# Web Scraping: Selenium

En este notebook veremos cómo automatizar la extracción de datos de una página web simulando la navegación. Para ello utilizaremos la librería Selenium:
https://selenium-python.readthedocs.io/

El software libre Selenium es un **framework** para realizar test automatizados de software a aplicaciones web. En principio, fue desarrollado para poner a prueba páginas y apps web, pero el **WebDriver** de Selenium también puede usarse con Python para realizar scraping. Si bien Selenium en sí no está escrito en Python, con este lenguaje de programación es posible acceder a las funciones del software.

A diferencia de Scrapy y de BeautifulSoup, Selenium no trabaja con el texto fuente en HTML de la web en cuestión, sino que carga la página en un navegador sin interfaz de usuario. El navegador interpreta entonces el código fuente de la página y crea, a partir de él, un **Document Object Model** (modelo de objetos de documento o DOM). Esta interfaz estandarizada permite poner a prueba las interacciones de los usuarios. De esta forma se consigue, por ejemplo, simular clics y rellenar formularios automáticamente. Los cambios en la web que resultan de dichas acciones se reflejan en el DOM. La estructura del proceso de web scraping con Selenium es la siguiente:

URL → Solicitud HTTP → HTML → Selenium → DOM

In [91]:
# Imports

import pandas as pd
from selenium import webdriver

Deberemos descargar un archivo Chrome driver para selenium en https://chromedriver.chromium.org/downloads, es importante que compruebes la versión de Chrome que utilizas y descargaues el driver correspondiente.
Una vez descargado crearemos una variable con la dirección completa del path al chromedriver 

El objeto driver es con el que trabajaremos a partir de ahora

In [92]:
# especificamos el path hasta nuestro driver recién descargado:
chrome_driver_path = 'C:\\Users\\LeoFr\\Desktop\\BootCamp\\chromedriver_win32\\chromedriver.exe'
options = webdriver.ChromeOptions()
# options.add_argument('headless')
# headless nos permite no tener que ver la navegación en la ventana de Chrome (TRABAJO A CIEGAS)

In [93]:
# Creamos el driver con el que nos vamos a manejar en la sesión de scrapeo:

driver = webdriver.Chrome(executable_path = chrome_driver_path, options = options)

In [94]:
# indicamos la URL de la página web a la que queremos acceder:

url = 'https://www.filmaffinity.com/es/main.html'

# el objeto driver nos va a permitir alterar el estado del la página

driver.get(url)


¡Vaya! Nos hemos encontrado con un pop-up que nos pide aceptar cookies o algo po el estilo, si nos lo queremos saltar, para poder automatizar el proceso deberemos:

1. Buscamos el botón
2. Hacemos click en el botón

Selenium nos permite buscar elementos por etiqueteas de html: https://selenium-python.readthedocs.io/locating-elements.html

CUIDADO! Los metodos de busqueda de elementos estan "duplicados". Tenemos *find_element_by_tag_name* y *find_elements_by_tag_name* el primero nos devolverá el primer elemento que encuentre (aunque haya más), el segundo nos devolverá una lista con todos los elementos que encuentre (aunque solo sea uno)

In [95]:
# Navegando en filmaffinity, en el pop-up:

elements_by_tag = driver.find_elements_by_tag_name('button') # Este me da todos los botones del push-up
element_by_class_name = driver.find_elements_by_class_name('css-47sehv') # Este accede al boton "ACEPTO" por class_name
elements_by_xpath = driver.find_element_by_xpath('/html/body/div[1]/div/div/div/div[2]/div/button[2]') # Este accede DIRECTAMENTE al botón, copiando su Xpath


Una vez tenemos los elementos podemos hacer varias cosas con ellos

Podemos extraer todos los atributos que tenga

In [96]:

# obtenemos todos sus métodos y atributos:

Podemos evaluar que tipo de elemento es (tag)

Podemos sacar el valor que tiene (el texto)

In [97]:
elements_by_xpath.text

'ACEPTO'

In [98]:
for valor in elements_by_tag:
    print(valor.text)

socios
MÁS OPCIONES
ACEPTO


In [99]:
# lo mismo con list comprehension:

[elem.text for elem in elements_by_tag]

['socios', 'MÁS OPCIONES', 'ACEPTO']

Incluso podemos guardar una imagen del elemento

In [100]:
# Vemos que es tipo 'WebElement' y en la documentación podremos encontrar sus métodos

type(elements_by_xpath)

selenium.webdriver.remote.webelement.WebElement

In [101]:
# guardamos como 'mi_imagen.png' la imagen asociada al xpath

elements_by_xpath.screenshot('mi_imagen.png')

True

Evaluamos que elementos hemos encontrado por el tag:

In [102]:
for index, element in enumerate(elements_by_tag):
    print('Elemento:', index)
    print('Texto del elemento:', index, 'es', element.text)
    print('El tag del elemento:', index, 'es', element.tag_name)
    element.screenshot('mi_imagen' + str(index) + '.png')

Elemento: 0
Texto del elemento: 0 es socios
El tag del elemento: 0 es button
Elemento: 1
Texto del elemento: 1 es MÁS OPCIONES
El tag del elemento: 1 es button
Elemento: 2
Texto del elemento: 2 es ACEPTO
El tag del elemento: 2 es button


Nos quedamos con el botón que nos interesa

In [103]:
boton_aceptar = elements_by_tag[2]
boton_aceptar

<selenium.webdriver.remote.webelement.WebElement (session="1b80c68ee506edf70cb828f418313285", element="2a6c0f2d-e3fc-4bbc-94b6-a2b251c80a8b")>

Si el elemento es interactivo podremos hacer más cosas además de las anteriores. Por ejemplo: hacer click

In [104]:
boton_aceptar.click()

Buscamos una película por título

In [105]:
from selenium.webdriver.common.keys import Keys

In [106]:
buscador = driver.find_element_by_xpath('/html/body/div[2]/div[1]/div/div[2]/form/input[1]')

In [107]:
buscador.send_keys('Interstellar')



In [108]:
# una vez escrita la búsqueda deberíamos poder activarla:
buscador.send_keys(Keys.ENTER)

In [109]:
# volvemos a la página anterior
driver.back()

### Vamos a buscar todas las películas que se estrenan el próximo viernes

1. Cogemos los containers que hay en la zona central

In [110]:
menu_lateral = driver.find_element_by_id('lsmenu')
menu_lateral

<selenium.webdriver.remote.webelement.WebElement (session="1b80c68ee506edf70cb828f418313285", element="91fc1612-355b-4913-9700-070e5e43cf5f")>

In [111]:
secciones_menu = menu_lateral.find_elements_by_tag_name('a')
secciones_menu

[<selenium.webdriver.remote.webelement.WebElement (session="1b80c68ee506edf70cb828f418313285", element="20bf13ec-959e-4ce4-953a-c23c3ea9b4cb")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1b80c68ee506edf70cb828f418313285", element="2edc08f6-2ab6-490f-9c2a-c90298c8d83e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1b80c68ee506edf70cb828f418313285", element="b25bed0c-a15f-41ea-a472-da74fcb4def7")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1b80c68ee506edf70cb828f418313285", element="a82f655b-7f6d-47ca-b94a-9e0b79d6677a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1b80c68ee506edf70cb828f418313285", element="9425456c-7b2d-47ce-a4e1-4270d8edf5ec")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1b80c68ee506edf70cb828f418313285", element="677bedc7-bf07-454f-92b2-284653ce8d9d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1b80c68ee506edf70cb828f418313285", element="d8496481-d38b-40ce-812f-a7

2. Vemos con cuál nos tenemos que quedar

In [112]:
for a in secciones_menu:
    if a.text == 'Próximos estrenos':
        a.click()
        break
        

In [ ]:
# [a for a in menu_lateral.find_elements_by_tag_name('a') if a.text == "Próximos estrenos"]    

# Este hace lo mismo pero sin click... lo paso Clara

Accedemos al container central, en el que aparecen los estrenos por semana que queremos ver, exactamente igual que hemos hecho antes

In [113]:
cajon_central = driver.find_elements_by_id("main-wrapper-rdcat")
for semana in cajon_central:
    print(semana.find_element_by_tag_name('div').text)
    print(semana.find_element_by_tag_name('div').get_attribute('id'))

21 de mayo de 2021
2021-05-21
28 de mayo de 2021
2021-05-28
3 de junio de 2021
2021-06-03
4 de junio de 2021
2021-06-04
11 de junio de 2021
2021-06-11
16 de junio de 2021
2021-06-16
18 de junio de 2021
2021-06-18
25 de junio de 2021
2021-06-25


In [114]:
for semana in cajon_central:
    fecha = semana.find_element_by_tag_name('div').get_attribute('id')
    if fecha == '2021-05-21':
        break

In [117]:
caratulas = semana.find_elements_by_class_name('mc-poster')
lista_pelis = []
for peli in caratulas:
    lista_pelis.append(peli.find_element_by_tag_name('a').get_attribute('href'))
lista_pelis

['https://www.filmaffinity.com/es/film637149.html',
 'https://www.filmaffinity.com/es/film478891.html',
 'https://www.filmaffinity.com/es/film713795.html',
 'https://www.filmaffinity.com/es/film112106.html',
 'https://www.filmaffinity.com/es/film329087.html',
 'https://www.filmaffinity.com/es/film950177.html',
 'https://www.filmaffinity.com/es/film388427.html',
 'https://www.filmaffinity.com/es/film132801.html',
 'https://www.filmaffinity.com/es/film759533.html',
 'https://www.filmaffinity.com/es/film407520.html',
 'https://www.filmaffinity.com/es/film452148.html',
 'https://www.filmaffinity.com/es/film158082.html',
 'https://www.filmaffinity.com/es/film108823.html']

Buscamos cómo acceder a las películas

Una vez tenemos todas las urls vamos a ver qué hacemos con cada una de ellas

In [120]:
# Accedemos a la página de la primera pelicula

driver.get(lista_pelis[0])

Vamos a ver el proceso que deberíamos hacer con cada una de las películas:

1. Sacamos toda la información que nos interesa

In [ ]:
# titulo, nota, numero de votos y ficha técnica


2. Creamos una lista a partir de la ficha técnica

In [ ]:
# Los nombres estan con tag  = 'dt' y los valores con 'dd'


3. Creamos un dataframe con la info

Ahora vamos a crear una función que nos haga todo esto para cada una de las películas:

## Modo Dios: moviendonos entre ventanas

Vamos a ver cómo nos podemos mover entre ventanas del navegador

Abrir nueva ventana:

Movernos a otra ventana

Cerrar ventana

Una vez cerramos la ventana tenemos que indicarle a qué ventana tiene que ir

Sabiendo cómo podemos movernos por entre las ventanas y sabiendo cómo extraer de cada página toda la información que necesitamos vamos a crear nuestro dataframe:

In [ ]:
# para abrir todos los links en lista_pelis


In [ ]:
# Creamos un dataframe con todas las pelis que se estrenan la próxima semana:


¡Tachán! Ya tenemos un dataframe con todas las películas que se van a estrenar el próximo viernes